In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_log_error
from tensorflow.keras.models import load_model

In [27]:
df = pd.read_csv(r'/home/long/longdata/kaggle compe/dataset/num_df.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   pickup_longitude   1458644 non-null  float64
 1   pickup_latitude    1458644 non-null  float64
 2   dropoff_longitude  1458644 non-null  float64
 3   dropoff_latitude   1458644 non-null  float64
 4   trip_duration      1458644 non-null  int64  
 5   haversine_km       1458644 non-null  float64
 6   bearing            1458644 non-null  float64
 7   pickup_cluster     1458644 non-null  int64  
 8   dropoff_cluster    1458644 non-null  int64  
dtypes: float64(6), int64(3)
memory usage: 100.2 MB


In [32]:
x = df.drop('trip_duration', axis=1)
y = df['trip_duration']
print(x.columns)

Index(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'haversine_km', 'bearing', 'pickup_cluster',
       'dropoff_cluster'],
      dtype='object')


In [33]:
df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,haversine_km,bearing,pickup_cluster,dropoff_cluster
0,-73.982155,40.767937,-73.964630,40.765602,455,1.498521,99.970196,0,8
1,-73.980415,40.738564,-73.999481,40.731152,663,1.805507,242.846232,0,9
2,-73.979027,40.763939,-74.005333,40.710087,2124,6.385098,200.319835,0,9
3,-74.010040,40.719971,-74.012268,40.706718,429,1.485498,187.262300,4,9
4,-73.973053,40.793209,-73.972923,40.782520,435,1.188588,179.473585,3,8


In [34]:
def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.clip_by_value(y_true, 1e-7, tf.reduce_max(y_true))
    y_pred = tf.clip_by_value(y_pred, 1e-7, tf.reduce_max(y_pred))
    log_true = tf.math.log(y_true + 1.0)
    log_pred = tf.math.log(y_pred + 1.0)
    return tf.sqrt(tf.reduce_mean(tf.square(log_pred - log_true)))

In [35]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [36]:
model = keras.Sequential([
    layers.Input(shape=(8,)), 

    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),

    layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=rmsle,
    metrics=[rmsle]
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [40]:
history = model.fit(
    x_scaled, y,
    validation_split=0.01,
    epochs=100,
    batch_size=256,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 0.4655 - rmsle: 0.4655 - val_loss: 0.4837 - val_rmsle: 0.4837
Epoch 2/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.4638 - rmsle: 0.4638 - val_loss: 0.4605 - val_rmsle: 0.4605
Epoch 3/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - loss: 0.4648 - rmsle: 0.4648 - val_loss: 0.4526 - val_rmsle: 0.4525
Epoch 4/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.4661 - rmsle: 0.4661 - val_loss: 0.4560 - val_rmsle: 0.4560
Epoch 5/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.4627 - rmsle: 0.4627 - val_loss: 0.4806 - val_rmsle: 0.4805
Epoch 6/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.4627 - rmsle: 0.4627 - val_loss: 0.4893 - val_rmsle: 0.4893
Epoch 7/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 0.4638 - rmsle: 0.4638 - val_loss: 0.4531 - val_rmsle: 0.4531
Epoch 8/100
5641/5641 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - loss: 0.4654 - rmsle: 0.4654 - val_loss: 0.4991 - val_rmsle:

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training RMSLE')
plt.plot(history.history['val_loss'], label='Validation RMSLE')
plt.title('Model RMSLE Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('RMSLE')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
model.save('bully-maguire.keras')

In [ ]:
test_df = pd.read_csv(r'/home/long/longdata/kaggle compe/dataset/test.csv')
df = pd.read_csv(r'/home/long/longdata/kaggle compe/dataset/num_df.csv')
scaler = StandardScaler()
test_df.head()

In [ ]:
# test_df = test_df.drop(['id', 'vendor_id', 'passenger_count', 'store_and_fwd_flag'], axis=1)
# test_df.head()

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371 
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def bearing(lat1, lon1, lat2, lon2):
    dLon = np.radians(lon2 - lon1)
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    x = np.sin(dLon) * np.cos(lat2)
    y = np.cos(lat1)*np.sin(lat2) - np.sin(lat1)*np.cos(lat2)*np.cos(dLon)
    return (np.degrees(np.arctan2(x, y)) + 360) % 360

test_df['pickup_datetime'] = pd.to_datetime(test_df['pickup_datetime'])
test_df['haversine_km'] = haversine(test_df['pickup_latitude'], test_df['pickup_longitude'],
                                    test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['bearing'] = bearing(test_df['pickup_latitude'], test_df['pickup_longitude'],
                             test_df['dropoff_latitude'], test_df['dropoff_longitude'])
test_df['trip_duration_minutes'] = test_df['trip_duration'] / 60
test_df['log_trip_duration'] = np.log1p(test_df['trip_duration'])
test_df['avg_speed_kph'] = test_df['haversine_km'] / (test_df['trip_duration_minutes'] / 60 + 1e-5)
test_df['is_zero_distance'] = test_df['haversine_km'] < 0.01
pickup_coords = df[['pickup_latitude', 'pickup_longitude']].dropna()
dropoff_coords = df[['dropoff_latitude', 'dropoff_longitude']].dropna()

pickup_kmeans = KMeans(n_clusters=10, random_state=42).fit(pickup_coords)
dropoff_kmeans = KMeans(n_clusters=10, random_state=42).fit(dropoff_coords)

test_df['pickup_cluster'] = pickup_kmeans.predict(test_df[['pickup_latitude', 'pickup_longitude']])
test_df['dropoff_cluster'] = dropoff_kmeans.predict(test_df[['dropoff_latitude', 'dropoff_longitude']])

feature_columns = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude',
                   'haversine_km', 'bearing', 'pickup_cluster', 'dropoff_cluster']
scaler = StandardScaler()
scaler.fit(df[feature_columns])  

x_test_scaled = scaler.transform(test_df[feature_columns])  
def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true = tf.clip_by_value(y_true, 1e-7, tf.reduce_max(y_true))
    y_pred = tf.clip_by_value(y_pred, 1e-7, tf.reduce_max(y_pred))
    log_true = tf.math.log(y_true + 1.0)
    log_pred = tf.math.log(y_pred + 1.0)
    return tf.sqrt(tf.reduce_mean(tf.square(log_pred - log_true)))

model = load_model(
    r'/home/long/longdata/kaggle compe/nyc-taxi-trip-duration/bully-maguire.keras',
    custom_objects={'rmsle': rmsle}
)
y_pred = model.predict(x_test_scaled)

test_predictions = pd.DataFrame({
    'predictions': y_pred.flatten()
})

test_predictions.to_csv('test_pred.csv', index=False)